In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import graphviz
from sklearn.model_selection import train_test_split
from sklearn import tree

In [6]:
df = pd.read_csv("task10_dataset.csv")

In [7]:
df.head()

,IDINT,IDPAC,NrInt,Sex,VAn,VLu,VNorm,Jud,AnEx,LuEx,...,ULCER3,ULCER4,ULCER5,ULCER6,STOMUCCORP,STOMUCCORP2,STOMUCCORP3,STOMUCCORP4,STOMUCCORP5,STOMUCCORP6
0,2958,3,1,F,12,6,12.5,CJ,2006,1,...,NaN,NaN,NaN,NaN,N,N,NaN,NaN,NaN,NaN
1,3116,3,2,F,12,6,12.5,CJ,2006,5,...,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN
2,2649,8,1,F,18,0,18.0,CJ,2005,1,...,NU,NaN,NaN,NaN,N,N,N,NaN,NaN,NaN
3,2813,8,2,F,18,0,18.0,CJ,2005,6,...,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN
4,3718,8,3,F,18,0,18.0,CJ,2008,3,...,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN


In [8]:
df.columns

Index(['IDINT', 'IDPAC', 'NrInt', 'Sex', 'VAn', 'VLu', 'VNorm', 'Jud', 'AnEx',
       'LuEx',
       ...
       'ULCER3', 'ULCER4', 'ULCER5', 'ULCER6', 'STOMUCCORP', 'STOMUCCORP2',
       'STOMUCCORP3', 'STOMUCCORP4', 'STOMUCCORP5', 'STOMUCCORP6'],
      dtype='object', length=112)

In [9]:
columns = (df.columns).to_numpy()
columns

array(['IDINT', 'IDPAC', 'NrInt', 'Sex', 'VAn', 'VLu', 'VNorm', 'Jud',
       'AnEx', 'LuEx', 'SDA', 'SDA2', 'SDA3', 'SDA4', 'SDA5', 'SDA6',
       'SDE', 'SDE2', 'SDE3', 'SDE4', 'SDE5', 'SDE6', 'VAR', 'VAR2',
       'VAR3', 'VAR4', 'VAR5', 'VAR6', 'HIPO', 'HIPO2', 'HIPO3', 'HIPO4',
       'HIPO5', 'HIPO6', 'ANEMIE', 'ANEMIE2', 'ANEMIE3', 'ANEMIE4',
       'ANEMIE5', 'ANEMIE6', 'AA', 'AA2', 'AA3', 'AA4', 'AA5', 'AA6',
       'HDSI', 'HDSI2', 'HDSI3', 'HDSI4', 'HDSI5', 'HDSI6', 'GAS', 'GAS2',
       'GAS3', 'GAS4', 'GAS5', 'GAS6', 'GASHP', 'GASHP2', 'GASHP3',
       'GASHP4', 'GASHP5', 'GASHP6', 'ESOFAGITA', 'ESOFAGITA2',
       'ESOFAGITA3', 'ESOFAGITA4', 'ESOFAGITA5', 'ESOFAGITA6', 'ANO',
       'ANO2', 'ANO3', 'ANO4', 'ANO5', 'ANO6', 'BC', 'BC2', 'BC3', 'BC4',
       'BC5', 'BC6', 'CEFALEE', 'CEFALEE2', 'CEFALEE3', 'CEFALEE4',
       'CEFALEE5', 'CEFALEE6', 'CONS', 'CONS2', 'CONS3', 'CONS4', 'CONS5',
       'CONS6', 'DRE', 'DRE2', 'DRE3', 'DRE4', 'DRE5', 'DRE6', 'ULCER',
       'ULCE

In [10]:
no_consultations = 6
first_symptom_index = np.where(columns == "SDA")[0][0]
no_consultations, first_symptom_index

(6, 10)

In [47]:
patient_data = columns[1:10]
useless = ['NrInt','IDPAC', 'VAn', 'VLu', 'Jud', 'AnEx', 'LuEx']
for elem in useless:
    patient_data = patient_data[patient_data != elem]
patient_data

array(['Sex', 'VNorm'], dtype=object)

In [48]:
consults = np.empty([6, 17], dtype=object)

consults[0].shape

(17,)

In [49]:
consults = np.empty([6, 17], dtype=object)
for i in range(0, 6):
    cons_i = []
    for pos, val in enumerate(columns):
        if pos >= first_symptom_index and ((pos - first_symptom_index) % no_consultations == i):
            cons_i = np.append(cons_i, val)
    consults[i] = cons_i
    print(consults[i])

['SDA' 'SDE' 'VAR' 'HIPO' 'ANEMIE' 'AA' 'HDSI' 'GAS' 'GASHP' 'ESOFAGITA'
 'ANO' 'BC' 'CEFALEE' 'CONS' 'DRE' 'ULCER' 'STOMUCCORP']
['SDA2' 'SDE2' 'VAR2' 'HIPO2' 'ANEMIE2' 'AA2' 'HDSI2' 'GAS2' 'GASHP2'
 'ESOFAGITA2' 'ANO2' 'BC2' 'CEFALEE2' 'CONS2' 'DRE2' 'ULCER2'
 'STOMUCCORP2']
['SDA3' 'SDE3' 'VAR3' 'HIPO3' 'ANEMIE3' 'AA3' 'HDSI3' 'GAS3' 'GASHP3'
 'ESOFAGITA3' 'ANO3' 'BC3' 'CEFALEE3' 'CONS3' 'DRE3' 'ULCER3'
 'STOMUCCORP3']
['SDA4' 'SDE4' 'VAR4' 'HIPO4' 'ANEMIE4' 'AA4' 'HDSI4' 'GAS4' 'GASHP4'
 'ESOFAGITA4' 'ANO4' 'BC4' 'CEFALEE4' 'CONS4' 'DRE4' 'ULCER4'
 'STOMUCCORP4']
['SDA5' 'SDE5' 'VAR5' 'HIPO5' 'ANEMIE5' 'AA5' 'HDSI5' 'GAS5' 'GASHP5'
 'ESOFAGITA5' 'ANO5' 'BC5' 'CEFALEE5' 'CONS5' 'DRE5' 'ULCER5'
 'STOMUCCORP5']
['SDA6' 'SDE6' 'VAR6' 'HIPO6' 'ANEMIE6' 'AA6' 'HDSI6' 'GAS6' 'GASHP6'
 'ESOFAGITA6' 'ANO6' 'BC6' 'CEFALEE6' 'CONS6' 'DRE6' 'ULCER6'
 'STOMUCCORP6']


In [50]:
first_consult = np.append(patient_data, consults[0])
first_consult

array(['Sex', 'VNorm', 'SDA', 'SDE', 'VAR', 'HIPO', 'ANEMIE', 'AA',
       'HDSI', 'GAS', 'GASHP', 'ESOFAGITA', 'ANO', 'BC', 'CEFALEE',
       'CONS', 'DRE', 'ULCER', 'STOMUCCORP'], dtype=object)

In [56]:
first_cons_df = df[first_consult]
first_x = first_cons_df.iloc[:,:-1]   # features - everything but the label
first_y = first_cons_df.iloc[:,-1:]   # label - last column

In [57]:
first_x

,Sex,VNorm,SDA,SDE,VAR,HIPO,ANEMIE,AA,HDSI,GAS,GASHP,ESOFAGITA,ANO,BC,CEFALEE,CONS,DRE,ULCER
0,F,12.50,NU,DA,NU,DA,NU,NU,NU,NU,NU,NU,NU,NU,NU,NU,NU,NU
1,F,12.50,NU,NU,NU,DA,NU,NU,NU,NU,NU,NU,NU,NU,NU,NU,NU,NU
2,F,18.00,NU,NU,NU,NU,NU,NU,NU,NU,NU,NU,NU,NU,NU,NU,NU,NU
3,F,18.00,NU,NU,NU,NU,NU,NU,NU,NU,NU,NU,NU,NU,NU,NU,NU,NU
4,F,18.00,NU,NU,NU,NU,NU,NU,NU,NU,NU,NU,NU,NU,NU,NU,NU,NU
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
494,M,17.00,NU,DA,NU,NU,NU,NU,NU,NU,NU,NU,NU,NU,NU,NU,NU,NU
495,F,0.67,NU,NU,NU,DA,NU,NU,NU,NU,NU,NU,DA,NU,NU,NU,NU,NU
496,F,1.08,NU,NU,NU,NU,NU,NU,NU,DA,NU,NU,DA,NU,NU,NU,NU,NU
497,F,15.00,NU,NU,NU,NU,NU,NU,NU,NU,NU,NU,NU,NU,NU,NU,NU,NU


In [58]:
first_y

,STOMUCCORP
0,N
1,N
2,N
3,N
4,N
...,...
494,N
495,N
496,ATROFIC
497,N


In [59]:
print(first_x.shape, " ", first_y.shape)

(499, 18)   (499, 1)


In [60]:
train_x, test_x, train_y, test_y = train_test_split(first_x, first_y, test_size=0.25, random_state=42)

# I tried to stratify the train/test sets using first_y, but there was only one member for a certain class
# thus the stratification was impossible (sklearn threw an error)

In [61]:
clf = tree.DecisionTreeClassifier(criterion="entropy", random_state=43)

In [63]:
clf.fit(first_x, first_y)

ValueError: could not convert string to float: 'F'